Creating a web scrapper to pull yeast strain information from the Brewing Science Institute Website

In [1]:
#pip install gazpacho

In [2]:
# Loading in all the necessary packages
from gazpacho import get, Soup
import numpy
import pandas as pd
import requests, bs4
import re, os
import time

In [3]:
# Verifying that the page request is valid
from gazpacho import get, Soup

url = 'https://brewingscience.com/product/ringwood-ale//'

# fetch step
html = get(url)

# extract step
soup = Soup(html)

Inspect the HTML Code on a web browser

![title](data/inspect.png)


In [4]:
# use find() to parse a specific HTML
gaz_data = soup.find('div', {'class':'container'})
gaz_data

[<div class="vc_col-sm-6 wpb_column vc_column_container"><div class="vc_column-inner"><div class="wpb_wrapper"><div class="w-post-elm product_gallery"><div class="woocommerce-product-gallery woocommerce-product-gallery--with-images woocommerce-product-gallery--columns-4 images" data-columns="4" style="opacity: 0; transition: opacity .25s ease-in-out;">
 	<div class="woocommerce-product-gallery__wrapper">
 		<div data-thumb="https://brewingscience.com/wp-content/uploads/2018/12/1187-Ringwood-Ale-Ale-Icon-150x150.png" data-thumb-alt="" class="woocommerce-product-gallery__image"><img width="500" height="500" src="https://brewingscience.com/wp-content/uploads/2018/12/1187-Ringwood-Ale-Ale-Icon.png" class="wp-post-image" alt="" title="1187-Ringwood-Ale-Ale-Icon" data-caption="" data-src="https://brewingscience.com/wp-content/uploads/2018/12/1187-Ringwood-Ale-Ale-Icon.png" data-large_image="https://brewingscience.com/wp-content/uploads/2018/12/1187-Ringwood-Ale-Ale-Icon.png" data-large_image

### Pull The Strain Name

In [5]:
# Pull The Strain Name

def pull_strain_name(soup):
    strain_name = soup.find('h1', {'class':'w-post-elm post_title us_custom_064d9100 entry-title color_link_inherit'})
    
    x = str(strain_name)
    start = x.find('>')
    end = x.find('</h1>')
    strain = x[start+1:end]
    print(strain)
    return strain

In [6]:
pull_strain_name(soup)

1187 Ringwood Ale


'1187 Ringwood Ale'

### Pull The Description

In [7]:
# Pull and save the description

def pull_strain_description(soup):
    description = soup.find('div', {'class':'w-post-elm post_content'})
    print(description)
    x= str(description)
    start = x.find('<p>')
    end = x.find('</p>')
    description = x[start+2:end]
    # print(description)
    return description

In [8]:
pull_strain_description(soup)

<div class="w-post-elm post_content" itemprop="text">
  <p>A classic English strain best suited for malt forward beer styles. Finishes estery and fruity and can accentuate malt profiles utilizing traditional English malts. Suitable for American brown ale and English pale ale. This yeast has high oxygen requirements compared to other ale strains, and viability can decline quickly if stored for more than a few days.</p>
</div>


'>A classic English strain best suited for malt forward beer styles. Finishes estery and fruity and can accentuate malt profiles utilizing traditional English malts. Suitable for American brown ale and English pale ale. This yeast has high oxygen requirements compared to other ale strains, and viability can decline quickly if stored for more than a few days.'

### Get Stats on Strain

In [9]:
def pull_stats(soup):
    raw = soup.find('div', {'class':'w-post-elm product_field attributes test-grid-attributes'})
    x= str(raw)
    print(x)

    fs = x.find('https://brewingscience.com/flocculation/') + 55
    fe = x.find('https://brewingscience.com/flocculation/') + 69
    flocculation = x[fs:fe]
    # print(temp)
    
    attens = x.find('brewingscience.com/attenuation') + 30
    attene = x.find('brewingscience.com/attenuation') + 39
    attenuation = x[attens:attene]
    # print(attenuation)

    temps = x.find('Temp Range:') + 80
    tempe = x.find('Temp Range:') + 90
    temp = x[temps:tempe]
    # print(temp)

    abve = x.find('"w-post-elm-before">ABV') + 93
    abvs = x.find('"w-post-elm-before">ABV') + 98
    abv = x[abve:abvs]
    # print(abv)
    
    sts = x.find('>STA1: </span>') + 75
    ste = x.find('>STA1: </span>') + 85
    STA1 = x[sts:ste]

    return(flocculation, attenuation, abv, temp, STA1)

In [10]:
pull_stats(soup)

<div class="w-post-elm product_field attributes test-grid-attributes">
  <div class="woocommerce-product-attributes-item--attribute_pa_attenuation">
    <span class="w-post-elm-before">Attenuation: </span>
    <span class="woocommerce-product-attributes-item__value">
      <a href="https://brewingscience.com/attenuation/69-73/" rel="tag">69%-73%</a>
    </span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_temp-range">
    <span class="w-post-elm-before">Temp Range: </span>
    <span class="woocommerce-product-attributes-item__value">64-74 °F</span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_flocculation">
    <span class="w-post-elm-before">Flocculation: </span>
    <span class="woocommerce-product-attributes-item__value">
      <a href="https://brewingscience.com/flocculation/high/" rel="tag">High</a>
    </span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_abv">
    <span class="w-post-elm-before">A

('">High</a>\n   ', '/69-73/" ', '>10%<', '>64-74 °F<', '>Negative<')

### Pull all the urls

The urls were pulled using the program "Screaming Frog SEO Spirder" it indexes the entire website. I then coppied and pased that info into a csv file to create the links/

![title](data/screamingFrog.png)


In [14]:
# The value for the base of each url
main_url = 'https://brewingscience.com/product/'

In [15]:
# Reading in all the urls
links = pd.read_csv('data/BSI Links - Sheet1.csv')
links

,BSI Links,Unnamed: 1,Unnamed: 2,URLEnd,Unnamed: 4,Concatenation,Unnamed: 6,Full URL
0,https://brewingscience.com/product/,NaN,NaN,ringwood-ale,/,ringwood-ale/,NaN,https://brewingscience.com/product/ringwood-ale/
1,NaN,NaN,NaN,1868-pasteur-champagne,/,1868-pasteur-champagne/,NaN,https://brewingscience.com/product/1868-pasteu...
2,NaN,NaN,NaN,3470-german-lager,/,3470-german-lager/,NaN,https://brewingscience.com/product/3470-german...
3,NaN,NaN,NaN,german-ale,/,german-ale/,NaN,https://brewingscience.com/product/german-ale/
4,NaN,NaN,NaN,american-white-ale-2,/,american-white-ale-2/,NaN,https://brewingscience.com/product/american-wh...
...,...,...,...,...,...,...,...,...
118,NaN,NaN,NaN,weizen-ii,/,weizen-ii/,NaN,https://brewingscience.com/product/weizen-ii/
119,NaN,NaN,NaN,weizen,/,weizen/,NaN,https://brewingscience.com/product/weizen/
120,NaN,NaN,NaN,german-weizen,/,german-weizen/,NaN,https://brewingscience.com/product/german-weizen/
121,NaN,NaN,NaN,kolsch-3,/,kolsch-3/,NaN,https://brewingscience.com/product/kolsch-3/


In [16]:
url_ends = links.URLEnd.values.tolist()

In [17]:
# Creating a list of all the URLs
full_url = []

for i in url_ends:
    x = main_url + i
    full_url.append(x)
    print(x)

https://brewingscience.com/product/ringwood-ale
https://brewingscience.com/product/1868-pasteur-champagne
https://brewingscience.com/product/3470-german-lager
https://brewingscience.com/product/german-ale
https://brewingscience.com/product/american-white-ale-2
https://brewingscience.com/product/london-ale-iii
https://brewingscience.com/product/london-ale-i
https://brewingscience.com/product/british-ale-iii
https://brewingscience.com/product/british-ale-ii
https://brewingscience.com/product/old-german-ale
https://brewingscience.com/product/chico-ale
https://brewingscience.com/product/kolsch-2
https://brewingscience.com/product/london-ale-ii
https://brewingscience.com/product/timothy-taylor
https://brewingscience.com/product/american-microbrewery-ale
https://brewingscience.com/product/henley-on-thames-ale
https://brewingscience.com/product/scotch-ale
https://brewingscience.com/product/irish-ale-2
https://brewingscience.com/product/british-ale
https://brewingscience.com/product/whitbread-

In [18]:
# Testing out loading all the information that will be needed for the first URL
import pandas as pd

def loadTable(soup):
    strain = pull_strain_name(soup)
    #pullMetrics(soup)
    flocculation, attenuation, abv, temp, STA1 = pull_stats(soup)
    description = pull_strain_description(soup)
    entry = {'Strain': [strain],
        'Attenuation' : [attenuation],
        'Flocculation' : [flocculation],
        'Alcohol_Tolerance' : [abv],
        'Fermentation_Temperature' : [temp],
        'Description' : [description],
            "STA1" : [STA1]}
    df = pd.DataFrame(entry)
    return df

In [19]:
loadTable(soup)

1187 Ringwood Ale
<div class="w-post-elm product_field attributes test-grid-attributes">
  <div class="woocommerce-product-attributes-item--attribute_pa_attenuation">
    <span class="w-post-elm-before">Attenuation: </span>
    <span class="woocommerce-product-attributes-item__value">
      <a href="https://brewingscience.com/attenuation/69-73/" rel="tag">69%-73%</a>
    </span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_temp-range">
    <span class="w-post-elm-before">Temp Range: </span>
    <span class="woocommerce-product-attributes-item__value">64-74 °F</span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_flocculation">
    <span class="w-post-elm-before">Flocculation: </span>
    <span class="woocommerce-product-attributes-item__value">
      <a href="https://brewingscience.com/flocculation/high/" rel="tag">High</a>
    </span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_abv">
    <span class="w-

,Strain,Attenuation,Flocculation,Alcohol_Tolerance,Fermentation_Temperature,Description,STA1
0,1187 Ringwood Ale,"/69-73/""",""">High</a>\n",>10%<,>64-74 °F<,>A classic English strain best suited for malt...,>Negative<


In [20]:
df = loadTable(soup)
df1 = df
df1

1187 Ringwood Ale
<div class="w-post-elm product_field attributes test-grid-attributes">
  <div class="woocommerce-product-attributes-item--attribute_pa_attenuation">
    <span class="w-post-elm-before">Attenuation: </span>
    <span class="woocommerce-product-attributes-item__value">
      <a href="https://brewingscience.com/attenuation/69-73/" rel="tag">69%-73%</a>
    </span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_temp-range">
    <span class="w-post-elm-before">Temp Range: </span>
    <span class="woocommerce-product-attributes-item__value">64-74 °F</span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_flocculation">
    <span class="w-post-elm-before">Flocculation: </span>
    <span class="woocommerce-product-attributes-item__value">
      <a href="https://brewingscience.com/flocculation/high/" rel="tag">High</a>
    </span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_abv">
    <span class="w-

,Strain,Attenuation,Flocculation,Alcohol_Tolerance,Fermentation_Temperature,Description,STA1
0,1187 Ringwood Ale,"/69-73/""",""">High</a>\n",>10%<,>64-74 °F<,>A classic English strain best suited for malt...,>Negative<


In [21]:
# Looping through the list
for i in full_url:
  print(i)
  # fetch step
  html = get(i)

  # extract step
  soup = Soup(html)
  df = loadTable(soup)
  df1 = pd.concat([df1, df])

https://brewingscience.com/product/ringwood-ale
1187 Ringwood Ale
<div class="w-post-elm product_field attributes test-grid-attributes">
  <div class="woocommerce-product-attributes-item--attribute_pa_attenuation">
    <span class="w-post-elm-before">Attenuation: </span>
    <span class="woocommerce-product-attributes-item__value">
      <a href="https://brewingscience.com/attenuation/69-73/" rel="tag">69%-73%</a>
    </span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_temp-range">
    <span class="w-post-elm-before">Temp Range: </span>
    <span class="woocommerce-product-attributes-item__value">64-74 °F</span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_flocculation">
    <span class="w-post-elm-before">Flocculation: </span>
    <span class="woocommerce-product-attributes-item__value">
      <a href="https://brewingscience.com/flocculation/high/" rel="tag">High</a>
    </span>
  </div>
  <div class="woocommerce-product-attribu

A-18 London Ale III
<div class="w-post-elm product_field attributes test-grid-attributes">
  <div class="woocommerce-product-attributes-item--attribute_pa_attenuation">
    <span class="w-post-elm-before">Attenuation: </span>
    <span class="woocommerce-product-attributes-item__value">
      <a href="https://brewingscience.com/attenuation/71-75/" rel="tag">71%-75%</a>
    </span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_temp-range">
    <span class="w-post-elm-before">Temp Range: </span>
    <span class="woocommerce-product-attributes-item__value">64-74 °F</span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_flocculation">
    <span class="w-post-elm-before">Flocculation: </span>
    <span class="woocommerce-product-attributes-item__value">
      <a href="https://brewingscience.com/flocculation/medium/" rel="tag">Medium</a>
    </span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_abv">
    <span cla

A-56 Chico Ale
<div class="w-post-elm product_field attributes test-grid-attributes">
  <div class="woocommerce-product-attributes-item--attribute_pa_attenuation">
    <span class="w-post-elm-before">Attenuation: </span>
    <span class="woocommerce-product-attributes-item__value">
      <a href="https://brewingscience.com/attenuation/73-77/" rel="tag">73%-77%</a>
    </span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_temp-range">
    <span class="w-post-elm-before">Temp Range: </span>
    <span class="woocommerce-product-attributes-item__value">60-72 °F</span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_flocculation">
    <span class="w-post-elm-before">Flocculation: </span>
    <span class="woocommerce-product-attributes-item__value">
      <a href="https://brewingscience.com/flocculation/medium/" rel="tag">Medium</a>
    </span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_abv">
    <span class="w

A-75 Henley on Thames Ale
<div class="w-post-elm product_field attributes test-grid-attributes">
  <div class="woocommerce-product-attributes-item--attribute_pa_attenuation">
    <span class="w-post-elm-before">Attenuation: </span>
    <span class="woocommerce-product-attributes-item__value">
      <a href="https://brewingscience.com/attenuation/72-76/" rel="tag">72%-76%</a>
    </span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_temp-range">
    <span class="w-post-elm-before">Temp Range: </span>
    <span class="woocommerce-product-attributes-item__value">62-72 °F</span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_flocculation">
    <span class="w-post-elm-before">Flocculation: </span>
    <span class="woocommerce-product-attributes-item__value">
      <a href="https://brewingscience.com/flocculation/medium/" rel="tag">Medium</a>
    </span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_abv">
    <sp

ABS3  Belgian Ale
<div class="w-post-elm product_field attributes test-grid-attributes">
  <div class="woocommerce-product-attributes-item--attribute_pa_attenuation">
    <span class="w-post-elm-before">Attenuation: </span>
    <span class="woocommerce-product-attributes-item__value">
      <a href="https://brewingscience.com/attenuation/72-76/" rel="tag">72%-76%</a>
    </span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_temp-range">
    <span class="w-post-elm-before">Temp Range: </span>
    <span class="woocommerce-product-attributes-item__value">65-75 °F</span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_flocculation">
    <span class="w-post-elm-before">Flocculation: </span>
    <span class="woocommerce-product-attributes-item__value">
      <a href="https://brewingscience.com/flocculation/high/" rel="tag">High</a>
    </span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_abv">
    <span class="w-

...

L-12 Common Lager
<div class="w-post-elm product_field attributes test-grid-attributes">
  <div class="woocommerce-product-attributes-item--attribute_pa_attenuation">
    <span class="w-post-elm-before">Attenuation: </span>
    <span class="woocommerce-product-attributes-item__value">
      <a href="https://brewingscience.com/attenuation/72-76/" rel="tag">72%-76%</a>
    </span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_temp-range">
    <span class="w-post-elm-before">Temp Range: </span>
    <span class="woocommerce-product-attributes-item__value">58-68 °F</span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_flocculation">
    <span class="w-post-elm-before">Flocculation: </span>
    <span class="woocommerce-product-attributes-item__value">
      <a href="https://brewingscience.com/flocculation/medium/" rel="tag">Medium</a>
    </span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_abv">
    <span class

L-47 Denmark Lager II
<div class="w-post-elm product_field attributes test-grid-attributes">
  <div class="woocommerce-product-attributes-item--attribute_pa_attenuation">
    <span class="w-post-elm-before">Attenuation: </span>
    <span class="woocommerce-product-attributes-item__value">
      <a href="https://brewingscience.com/attenuation/73-77/" rel="tag">73%-77%</a>
    </span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_temp-range">
    <span class="w-post-elm-before">Temp Range: </span>
    <span class="woocommerce-product-attributes-item__value">46-56 °F</span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_flocculation">
    <span class="w-post-elm-before">Flocculation: </span>
    <span class="woocommerce-product-attributes-item__value">
      <a href="https://brewingscience.com/flocculation/low/" rel="tag">Low</a>
    </span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_abv">
    <span class="

Norwegian Farmhouse l
<div class="w-post-elm product_field attributes test-grid-attributes">
  <div class="woocommerce-product-attributes-item--attribute_pa_attenuation">
    <span class="w-post-elm-before">Attenuation: </span>
    <span class="woocommerce-product-attributes-item__value">
      <a href="https://brewingscience.com/attenuation/75-82/" rel="tag">75%-82%</a>
    </span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_temp-range">
    <span class="w-post-elm-before">Temp Range: </span>
    <span class="woocommerce-product-attributes-item__value">65-75 °F</span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_flocculation">
    <span class="w-post-elm-before">Flocculation: </span>
    <span class="woocommerce-product-attributes-item__value">
      <a href="https://brewingscience.com/flocculation/medium/" rel="tag">Medium</a>
    </span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_abv">
    <span c

S-25 Bier de Garde
<div class="w-post-elm product_field attributes test-grid-attributes">
  <div class="woocommerce-product-attributes-item--attribute_pa_attenuation">
    <span class="w-post-elm-before">Attenuation: </span>
    <span class="woocommerce-product-attributes-item__value">
      <a href="https://brewingscience.com/attenuation/74-79/" rel="tag">74%-79%</a>
    </span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_temp-range">
    <span class="w-post-elm-before">Temp Range: </span>
    <span class="woocommerce-product-attributes-item__value">70-95 °F</span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_flocculation">
    <span class="w-post-elm-before">Flocculation: </span>
    <span class="woocommerce-product-attributes-item__value">
      <a href="https://brewingscience.com/flocculation/low/" rel="tag">Low</a>
    </span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_abv">
    <span class="w-p

W177 Kolsch
<div class="w-post-elm product_field attributes test-grid-attributes">
  <div class="woocommerce-product-attributes-item--attribute_pa_attenuation">
    <span class="w-post-elm-before">Attenuation: </span>
    <span class="woocommerce-product-attributes-item__value">
      <a href="https://brewingscience.com/attenuation/73-77/" rel="tag">73%-77%</a>
    </span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_temp-range">
    <span class="w-post-elm-before">Temp Range: </span>
    <span class="woocommerce-product-attributes-item__value">54-64 °F</span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_flocculation">
    <span class="w-post-elm-before">Flocculation: </span>
    <span class="woocommerce-product-attributes-item__value">
      <a href="https://brewingscience.com/flocculation/low/" rel="tag">Low</a>
    </span>
  </div>
  <div class="woocommerce-product-attributes-item--attribute_pa_abv">
    <span class="w-post-elm

In [22]:
df1

,Strain,Attenuation,Flocculation,Alcohol_Tolerance,Fermentation_Temperature,Description,STA1
0,1187 Ringwood Ale,"/69-73/""",""">High</a>\n",>10%<,>64-74 °F<,>A classic English strain best suited for malt...,>Negative<
0,1187 Ringwood Ale,"/69-73/""",""">High</a>\n",>10%<,>64-74 °F<,>A classic English strain best suited for malt...,>Negative<
0,1868 Pasteur Champagne,"/67-68/""",>Low</a>\n <,>18%<,>39-95 °F<,>The world’s most widely used champagne yeast....,>Negative<
0,3470 German Lager,"/73-78/""","ag"">Medium</a>",>11%<,>46-54 °F<,>The most widely used German lager strain in t...,>Negative<
0,A-07 German Ale,"/73-77/""",>Low</a>\n <,>11%<,>55-66 °F<,">From Germany, this top-cropping ale yeast fer...",>Negative<
...,...,...,...,...,...,...,...
0,W-33 Weizen II,"/70-76/""","ag"">Medium</a>",>10%<,>63-75 °F<,>A well-balanced German weizen strain that cre...,>Negative<
0,W-38 Weizen,"/73-77/""",>Low</a>\n <,>10%<,>64-70 °F<,>A German weizen yeast producing more esters a...,>Negative<
0,W-68 German Weizen,"/73-77/""",>Low</a>\n <,>10%<,>64-70 °F<,>A wonderful top-cropping German weizen ale ye...,>Negative<
0,W177 Kolsch,"/73-77/""",>Low</a>\n <,>10%<,>54-64 °F<,>The classic strain used for the production of...,>Negative<


# Saving All the Urls to a CSV File

In [23]:
df1.to_csv('BSI_Strains_2024.csv')